In [ ]:
import torch

-----

## Kernel

affinity_from_flow() and affinity_matrix_from_pointset_to_pointset()

In [ ]:
A = torch.Tensor([[0,0],[2,0],[1,1],[1,0]])
n = A.shape[0]
print(A)

tensor([[0., 0.],
        [2., 0.],
        [1., 1.],
        [1., 0.]])


In [ ]:
A1 = A.T.repeat(n,1,1)
A1

tensor([[[0., 2., 1., 1.],
         [0., 0., 1., 0.]],

        [[0., 2., 1., 1.],
         [0., 0., 1., 0.]],

        [[0., 2., 1., 1.],
         [0., 0., 1., 0.]],

        [[0., 2., 1., 1.],
         [0., 0., 1., 0.]]])

In [ ]:
A2 = A[:,:,None].repeat(1,1,n)
A2

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[2., 2., 2., 2.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [0., 0., 0., 0.]]])

In [ ]:
A3 = (A2-A1)
A3

tensor([[[ 0., -2., -1., -1.],
         [ 0.,  0., -1.,  0.]],

        [[ 2.,  0.,  1.,  1.],
         [ 0.,  0., -1.,  0.]],

        [[ 1., -1.,  0.,  0.],
         [ 1.,  1.,  0.,  1.]],

        [[ 1., -1.,  0.,  0.],
         [ 0.,  0., -1.,  0.]]])

In [ ]:
A3 = A3.transpose(1,2)
A3

tensor([[[ 0.,  0.],
         [-2.,  0.],
         [-1., -1.],
         [-1.,  0.]],

        [[ 2.,  0.],
         [ 0.,  0.],
         [ 1., -1.],
         [ 1.,  0.]],

        [[ 1.,  1.],
         [-1.,  1.],
         [ 0.,  0.],
         [ 0.,  1.]],

        [[ 1.,  0.],
         [-1.,  0.],
         [ 0., -1.],
         [ 0.,  0.]]])

In [ ]:
length_of_directions = torch.linalg.norm(A3,dim=-1)
length_of_directions

tensor([[0.0000, 2.0000, 1.4142, 1.0000],
        [2.0000, 0.0000, 1.4142, 1.0000],
        [1.4142, 1.4142, 0.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 0.0000]])

In [ ]:
import torch.nn.functional as F
normed_directions = F.normalize(A3,dim=-1)
normed_directions

tensor([[[ 0.0000,  0.0000],
         [-1.0000,  0.0000],
         [-0.7071, -0.7071],
         [-1.0000,  0.0000]],

        [[ 1.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.7071, -0.7071],
         [ 1.0000,  0.0000]],

        [[ 0.7071,  0.7071],
         [-0.7071,  0.7071],
         [ 0.0000,  0.0000],
         [ 0.0000,  1.0000]],

        [[ 1.0000,  0.0000],
         [-1.0000,  0.0000],
         [ 0.0000, -1.0000],
         [ 0.0000,  0.0000]]])

In [ ]:
flows = torch.tensor([[1,0],[1,0],[0,2],[1,0]]).float()
flows

tensor([[1., 0.],
        [1., 0.],
        [0., 2.],
        [1., 0.]])

In [ ]:
dot_products = normed_directions*flows
dot_products

tensor([[[ 0.0000,  0.0000],
         [-1.0000,  0.0000],
         [-0.0000, -1.4142],
         [-1.0000,  0.0000]],

        [[ 1.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000, -1.4142],
         [ 1.0000,  0.0000]],

        [[ 0.7071,  0.0000],
         [-0.7071,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 1.0000,  0.0000],
         [-1.0000,  0.0000],
         [ 0.0000, -2.0000],
         [ 0.0000,  0.0000]]])

In [ ]:
projections = dot_products.sum(-1)
projections
# rows = destination
# columns = origin

tensor([[ 0.0000, -1.0000, -1.4142, -1.0000],
        [ 1.0000,  0.0000, -1.4142,  1.0000],
        [ 0.7071, -0.7071,  0.0000,  0.0000],
        [ 1.0000, -1.0000, -2.0000,  0.0000]])

In [ ]:
flow_mag = torch.linalg.norm(flows,dim=1)
flow_mag

tensor([1., 1., 2., 1.])

In [ ]:
flow_mag_directions = flow_mag.repeat(n,1)
flow_mag_directions

tensor([[1., 1., 2., 1.],
        [1., 1., 2., 1.],
        [1., 1., 2., 1.],
        [1., 1., 2., 1.]])

In [ ]:
cost_from_flow = flow_mag_directions - projections
cost_from_flow = torch.abs(cost_from_flow)
cost_from_flow

tensor([[1.0000, 2.0000, 3.4142, 2.0000],
        [0.0000, 1.0000, 3.4142, 0.0000],
        [0.2929, 1.7071, 2.0000, 1.0000],
        [0.0000, 2.0000, 4.0000, 1.0000]])

In [ ]:
cost_with_flow = cost_from_flow + length_of_directions
cost_with_flow = cost_with_flow.T
cost_with_flow

tensor([[1.0000, 2.0000, 1.7071, 1.0000],
        [4.0000, 1.0000, 3.1213, 3.0000],
        [4.8284, 4.8284, 2.0000, 5.0000],
        [3.0000, 1.0000, 2.0000, 1.0000]])

In [ ]:
kernel = torch.exp(cost_with_flow)
kernel

tensor([[  2.7183,   7.3891,   5.5130,   2.7183],
        [ 54.5981,   2.7183,  22.6763,  20.0855],
        [125.0142, 125.0142,   7.3891, 148.4132],
        [ 20.0855,   2.7183,   7.3891,   2.7183]])

---

### MultiscaleDiffusionFlowEmbedder() Functions

diffusion_loss()